# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")

Mounted at /content/gdrive


In [2]:
%pip install mambapy datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch, transformers, importlib
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer
import Mamba, VAE, Dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

# Lab

In [11]:
%pip install causal-conv1d>=1.2.0
%pip install mamba-ssm
%pip install triton

In [13]:
%pip uninstall mamba-ssmT

Found existing installation: mamba-ssm 2.2.4
Uninstalling mamba-ssm-2.2.4:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/mamba_ssm-2.2.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/mamba_ssm/*
    /usr/local/lib/python3.10/dist-packages/selective_scan_cuda.cpython-310-x86_64-linux-gnu.so
Proceed (Y/n)? Y
  Successfully uninstalled mamba-ssm-2.2.4


In [14]:
!python Trainer.py

2025-01-07 08:12:22.113106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-07 08:12:22.134428: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-07 08:12:22.140890: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-07 08:12:23.499257: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
The fast path is not available because one of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the mamba.py backend. To install follow https://github.com/state-spa

# Check

In [3]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Sunday.", "Today is."],
                      return_tensors="pt",
                      truncation=True,
                      max_length=128,
                      padding="max_length") #.to('cuda')
ones = torch.ones((inputs['attention_mask'].size(0), 1))
inputs['attention_mask'] = torch.cat([ones, inputs['attention_mask'][:, :-1]], dim=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [18]:
importlib.reload(VAE)

model = VAE.MambaVAE().cuda()
model.train()
model(inputs['input_ids'].cuda(), inputs['attention_mask'].cuda(), do_sample=False)

(tensor(27.2230, device='cuda:0', grad_fn=<DivBackward0>),
 tensor(40509.1445, device='cuda:0', grad_fn=<MulBackward0>),
 tensor([[[ 15.4702,  -6.4229,  11.7433,  ...,  -6.3039,  -6.8388,  -6.4293],
          [ 13.2941, -49.8134,   4.4575,  ..., -49.5522, -50.0998, -49.6902],
          [  9.3128, -42.6582,   0.5586,  ..., -42.3826, -42.9728, -42.5656],
          ...,
          [ -0.4556,   1.9405,  -0.7334,  ...,   2.2527,   1.9329,   1.9330],
          [  1.4255,   0.6849,   0.5462,  ...,   0.9937,   0.7871,   0.7586],
          [ -0.3228,   1.0471,   1.2121,  ...,   1.2998,   1.3047,   1.2445]],
 
         [[ 15.4759,  -6.3992,  11.7435,  ...,  -6.2798,  -6.8146,  -6.4048],
          [ 13.3054, -49.7845,   4.4584,  ..., -49.5228, -50.0701, -49.6606],
          [  9.3230, -42.6298,   0.5528,  ..., -42.3538, -42.9437, -42.5368],
          ...,
          [ -0.4556,   1.9405,  -0.7334,  ...,   2.2527,   1.9329,   1.9330],
          [  1.4255,   0.6849,   0.5462,  ...,   0.9937,   0.7871,

In [ ]:
importlib.reload(Mamba)
import Mamba

config = MambaConfig.from_pretrained("state-spaces/mamba-130m-hf")
config.use_mambapy = True
# config.num_hidden_layers = 12
model1 = Mamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config) #.to('cuda')
model1.eval()

res2 = model1(**inputs, output_ssm_last_states = True)

In [ ]:
ssm_last_states = res2.ssm_last_states

In [ ]:
res2 = model1(**inputs, inputs_ssm_states=ssm_last_states, output_ssm_last_states = True)
# print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))
# print(res2.ssm_last_states)

In [ ]:
torch.abs(res1.ssm_last_states - res2.ssm_last_states).sum()

tensor(0.0430, grad_fn=<SumBackward0>)

In [12]:
config = MambaConfig.from_pretrained("state-spaces/mamba-130m-hf")
config.use_mambapy = True
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config).cuda()
model2.train()

MambaForCausalLM(
  (backbone): MambaModel(
    (embeddings): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x MambaBlock(
        (norm): MambaRMSNorm(768, eps=1e-05)
        (mixer): MambaMixer(
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
      )
    )
    (norm_f): MambaRMSNorm(768, eps=1e-05)
  )
  (lm_head): Linear(in_features=768, out_features=50280, bias=False)
)

In [13]:
model2(inputs['input_ids'].cuda(), inputs['attention_mask'].cuda()).logits

tensor([[[-56.7010, -72.5961, -54.6084,  ..., -72.7466, -72.4688, -72.7352],
         [-60.5220, -74.4318, -57.2212,  ..., -74.6039, -74.3563, -74.6040],
         [-51.9266, -66.3724, -47.7203,  ..., -66.3502, -66.2736, -66.4541],
         ...,
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359]],

        [[-56.7010, -72.5961, -54.6084,  ..., -72.7466, -72.4688, -72.7352],
         [-60.5220, -74.4318, -57.2212,  ..., -74.6039, -74.3563, -74.6040],
         [-50.4484, -69.4465, -57.0026,  ..., -69.6075, -69.4996, -69.6404],
         ...,
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359],
         [273.4377, -29.3720,  61.1656,  ..., -29.2746, -29.3745, -29.4359]]],
       device='cuda:0', grad_fn=<UnsafeViewB

In [ ]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(**inputs).logits - model1(**inputs).logits).abs().sum())

loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
loop
tensor(0., grad_fn=<SumBackward0>)
